In [886]:
import numpy as np
import pandas as pd

In [887]:
df = pd.read_csv("inningHighlights.csv")
df.head()
df.shape

(238483, 5)

In [888]:
df = df[df["Inning"].isin(["T1", "B1"])]
df.head()

,Inning,Runs,Hits,Errors,Game
0,T1,0,0,0,360403123
1,B1,0,1,0,360403123
17,T1,2,2,0,360403130
18,B1,0,1,0,360403130
35,T1,0,0,0,360403107


In [889]:
df = df.groupby("Game")['Runs'].sum().reset_index()
df.head()


,Game,Runs
0,360403107,1
1,360403123,0
2,360403130,2
3,360404101,0
4,360404103,1


In [890]:
df['Runs'] = df['Runs'] < 1
df.head()
df.shape

(13379, 2)

In [891]:
df.rename(columns={'Runs': 'NRFI'}, inplace=True)
df.head()
df.shape

(13379, 2)

In [892]:
games = pd.read_csv('games.csv')
games.head()

,Game,away,away-record,awayaway-record,home,home-record,homehome-record,away-score,home-score,postseason info,Walks Issued - Away,Walks Issued - Home,Stolen Bases - Away,Stolen Bases - Home,Strikeouts Thrown - Away,Strikeouts Thrown - Home,Total Bases - Away,Total Bases - Home,Stadium,Date,Location,Odds,O/U,Attendance,Capacity,Duration,Umpires,WIN - Pitcher - Stats,WIN - Pitcher - Id,WIN - Pitcher - Name,WIN - Pitcher - AbbrName,WIN - Pitcher - Record,LOSS - Pitcher - Stats,LOSS - Pitcher - Id,LOSS - Pitcher - Name,LOSS - Pitcher - AbbrName,LOSS - Pitcher - Record,SAVE - Pitcher - Stats,SAVE - Pitcher - Id,SAVE - Pitcher - Name,SAVE - Pitcher - AbbrName,SAVE - Pitcher - Record,Extra Innings
0,360403123,STL,0-1,0-1 Away,PIT,1-0,1-0 Home,1.0,4.0,NaN,5.0,5.0,0.0,0.0,5.0,14.0,5.0,13.0,\n\n\n\n\n\t\t\t\t\t\t\tPNC Park\n\t\t\t\t\t\t...,2016-04-03T17:00Z,"Pittsburgh, Pennsylvania\n\t\t\t\t\t\t\t\n\t\t...",NaN,NaN,"39,500","38,362",3:02,"Home Plate Umpire - Jerry Layne, First Base Um...","6.0 IP, 0 ER, 10 K, 5 BB",6211.0,Francisco Liriano,F. Liriano,(1-0),"6.0 IP, 3 ER, 3 K, 3 BB",5403.0,Adam Wainwright,A. Wainwright,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
1,360403130,TOR,1-0,1-0 Away,TB,0-1,0-1 Home,5.0,3.0,NaN,1.0,3.0,0.0,0.0,7.0,16.0,11.0,11.0,\n\n\n\n\n\t\t\t\t\t\t\tTropicana Field\n\t\t\...,2016-04-03T20:00Z,"St. Petersburg, Florida\n\t\t\t\t\t\t\t\n\t\t\...",NaN,NaN,"31,042","31,042",2:51,"Home Plate Umpire - Mike Everitt, First Base U...","8.0 IP, 3 ER, 5 K, 1 BB",32815.0,Marcus Stroman,M. Stroman,(1-0),"5.0 IP, 2 ER, 12 K, 3 BB",31003.0,Chris Archer,C. Archer,(0-1),"1.0 IP, 0 ER, 2 K, 0 BB",32693.0,Roberto Osuna,R. Osuna,(1),NaN
2,360403107,NYM,0-1,0-1 Away,KC,1-0,1-0 Home,3.0,4.0,NaN,2.0,6.0,0.0,1.0,3.0,9.0,8.0,9.0,\n\n\n\n\n\t\t\t\t\t\t\tKauffman Stadium\n\t\t...,2016-04-04T00:30Z,"Kansas City, Missouri\n\t\t\t\t\t\t\t\n\t\t\t\...",NaN,NaN,"40,030","37,903",3:13,"Home Plate Umpire - Gerry Davis, First Base Um...","6.0 IP, 0 ER, 5 K, 3 BB",6401.0,Edinson Volquez,E. Volquez,(1-0),"5.2 IP, 3 ER, 2 K, 2 BB",31214.0,Matt Harvey,M. Harvey,(0-1),"1.0 IP, 0 ER, 2 K, 1 BB",28957.0,Wade Davis,W. Davis,(1),NaN
3,360404108,SF,1-0,1-0 Away,MIL,0-1,0-1 Home,12.0,3.0,NaN,5.0,5.0,1.0,0.0,10.0,3.0,30.0,14.0,\n\n\n\n\n\t\t\t\t\t\t\tMiller Park\n\t\t\t\t\...,2016-04-04T18:10Z,"Milwaukee, Wisconsin\n\t\t\t\t\t\t\t\n\t\t\t\t...",NaN,NaN,"44,318",NaN,3:35,"Home Plate Umpire - Brian Gorman, First Base U...","5.0 IP, 3 ER, 6 K, 5 BB",29949.0,Madison Bumgarner,M. Bumgarner,(1-0),"4.0 IP, 4 ER, 2 K, 2 BB",31140.0,Wily Peralta,W. Peralta,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
4,360404101,MIN,0-1,0-1 Away,BAL,1-0,1-0 Home,2.0,3.0,NaN,5.0,0.0,0.0,1.0,10.0,10.0,10.0,13.0,\n\n\n\n\n\t\t\t\t\t\t\tOriole Park at Camden ...,2016-04-04T19:05Z,"Baltimore, Maryland\n\t\t\t\t\t\t\t\n\t\t\t\t\...",NaN,NaN,"45,785","45,971",2:48,"Home Plate Umpire - Mike Winters, First Base U...","1.0 IP, 0 ER, 0 K, 0 BB",31054.0,Zack Britton,Z. Britton,(1-0),"0.2 IP, 1 ER, 1 K, 1 BB",29265.0,Kevin Jepsen,K. Jepsen,(0-1),NaN,NaN,NaN,NaN,NaN,NaN


In [893]:
df = pd.merge(df, games, on='Game', how='inner')
df.sort_values('Date', ascending=False).head()

,Game,NRFI,away,away-record,awayaway-record,home,home-record,homehome-record,away-score,home-score,postseason info,Walks Issued - Away,Walks Issued - Home,Stolen Bases - Away,Stolen Bases - Home,Strikeouts Thrown - Away,Strikeouts Thrown - Home,Total Bases - Away,Total Bases - Home,Stadium,Date,Location,Odds,O/U,Attendance,Capacity,Duration,Umpires,WIN - Pitcher - Stats,WIN - Pitcher - Id,WIN - Pitcher - Name,WIN - Pitcher - AbbrName,WIN - Pitcher - Record,LOSS - Pitcher - Stats,LOSS - Pitcher - Id,LOSS - Pitcher - Name,LOSS - Pitcher - AbbrName,LOSS - Pitcher - Record,SAVE - Pitcher - Stats,SAVE - Pitcher - Id,SAVE - Pitcher - Name,SAVE - Pitcher - AbbrName,SAVE - Pitcher - Record,Extra Innings
13387,401361981,True,ATL,88-73,46-35 Away,HOU,95-67,51-30 Home,7.0,0.0,"World Series - Game 6, ATL Wins series 4-2",0.0,3.0,0.0,0.0,10.0,9.0,17.0,6.0,\n\n\n\n\n\t\t\t\t\t\t\tMinute Maid Park\n\t\t...,2021-11-03T00:09Z,"Houston, Texas\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t...",Line: HOU -125,Over/Under: 9,"42,868","42,060",3:22,"Home Plate Umpire - Mike Muchlinski, First Bas...","6.0 IP, 0 ER, 6 K, 0 BB",32685.0,Max Fried,M. Fried,(1-1),"2.2 IP, 3 ER, 3 K, 1 BB",4684365.0,Luis Garcia,L. Garcia,(0-2),NaN,NaN,NaN,NaN,NaN,NaN
13386,401361980,False,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,9.0,5.0,"World Series - Game 5, ATL Wins series 4-2",2.0,6.0,1.0,0.0,7.0,9.0,15.0,15.0,\n\n\n\n\n\t\t\t\t\t\t\tTruist Park\n\t\t\t\t\...,2021-11-01T00:15Z,"Atlanta, Georgia\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t...",Line: HOU -125,Over/Under: 9,"43,122","41,149",4:00,"Home Plate Umpire - Ted Barrett, First Base Um...","1.0 IP, 0 ER, 0 K, 0 BB",35759.0,Jose Urquidy,J. Urquidy,(2-0),"1.0 IP, 3 ER, 2 K, 2 BB",36133.0,A.J. Minter,A.J. Minter,(1-1),NaN,NaN,NaN,NaN,NaN,NaN
13385,401361979,False,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,2.0,3.0,"World Series - Game 4, ATL Wins series 4-2",3.0,5.0,1.0,0.0,11.0,7.0,11.0,15.0,\n\n\n\n\n\t\t\t\t\t\t\tTruist Park\n\t\t\t\t\...,2021-10-31T00:09Z,"Atlanta, Georgia\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t...",Line: EVEN,Over/Under: 9,"43,125","41,149",3:45,"Home Plate Umpire - Dan Bellino, First Base Um...","1.0 IP, 0 ER, 0 K, 0 BB",31264.0,Tyler Matzek,T. Matzek,(1-0),"0.1 IP, 2 ER, 1 K, 0 BB",41261.0,Cristian Javier,C. Javier,(0-1),"1.0 IP, 0 ER, 1 K, 0 BB",31549.0,Will Smith,W. Smith,(2),NaN
13384,401361978,True,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,0.0,2.0,"World Series - Game 3, ATL Wins series 4-2",5.0,3.0,1.0,0.0,10.0,7.0,2.0,11.0,\n\n\n\n\n\t\t\t\t\t\t\tTruist Park\n\t\t\t\t\...,2021-10-30T00:09Z,"Atlanta, Georgia\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t...",Line: ATL -125,Over/Under: 9,"42,898","41,149",3:24,"Home Plate Umpire - Tom Hallion, First Base Um...","5.0 IP, 0 ER, 4 K, 3 BB",40916.0,Ian Anderson,I. Anderson,(1-0),"3.2 IP, 1 ER, 6 K, 4 BB",4684365.0,Luis Garcia,L. Garcia,(0-1),"1.0 IP, 0 ER, 0 K, 0 BB",31549.0,Will Smith,W. Smith,(1),NaN
13383,401361977,False,ATL,88-73,46-35 Away,HOU,95-67,51-30 Home,2.0,7.0,"World Series - Game 2, ATL Wins series 4-2",2.0,2.0,0.0,2.0,11.0,12.0,11.0,14.0,\n\n\n\n\n\t\t\t\t\t\t\tMinute Maid Park\n\t\t...,2021-10-28T00:09Z,"Houston, Texas\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t...",Line: HOU -121,Over/Under: 9,"42,833","42,060",3:11,"Home Plate Umpire - Ron Kulpa, First Base Umpi...","5.0 IP, 2 ER, 7 K, 0 BB",35759.0,Jose Urquidy,J. Urquidy,(1-0),"5.0 IP, 6 ER, 6 K, 1 BB",32685.0,Max Fried,M. Fried,(0-1),NaN,NaN,NaN,NaN,NaN,NaN


In [894]:
df = df[['Game','NRFI','away','home','Date']]
df.head()

,Game,NRFI,away,home,Date
0,360403107,False,NYM,KC,2016-04-04T00:30Z
1,360403123,True,STL,PIT,2016-04-03T17:00Z
2,360403130,False,TOR,TB,2016-04-03T20:00Z
3,360404101,True,MIN,BAL,2016-04-04T19:05Z
4,360404103,False,CHC,LAA,2016-04-05T02:00Z


In [895]:
df['year'] = pd.to_datetime(df['Date']).dt.year
df = df.drop(columns="Date")
df.head()
df.shape

(13395, 5)

In [896]:
pitchers = pd.read_csv('pitchersByGame.csv')
pitchers.head()

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_13818/310525727.py:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pitchers = pd.read_csv('pitchersByGame.csv')


,Pitchers,IP,H,R,ER,BB,K,HR,PC-ST,ERA,PC,Game,Team,Extra,Pitcher Id
0,A. Wainwright,6.0,6,3,3,3,3,0,96-59,4.5,96,360403123,STL,"(L,0-1)",5403
1,S. Oh,1.0,0,0,0,2,2,0,27-12,0.0,27,360403123,STL,NaN,34893
2,S. Maness,1.0,3,1,1,0,0,0,21-13,9.0,21,360403123,STL,NaN,32639
3,TEAM,8.0,9,4,4,5,5,0,144-84,NaN,144,360403123,STL,NaN,-
4,F. Liriano,6.0,3,0,0,5,10,0,94-50,0.0,94,360403123,PIT,"(W,1-0)",6211


In [897]:
pitchers = pitchers[["Pitchers", "Pitcher Id", 'Game', 'Team']]
pitchers.head(n=50)

,Pitchers,Pitcher Id,Game,Team
0,A. Wainwright,5403,360403123,STL
1,S. Oh,34893,360403123,STL
2,S. Maness,32639,360403123,STL
3,TEAM,-,360403123,STL
4,F. Liriano,6211,360403123,PIT
5,T. Watson,31513,360403123,PIT
6,N. Feliz,30149,360403123,PIT
7,M. Melancon,29446,360403123,PIT
8,TEAM,-,360403123,PIT
9,M. Stroman,32815,360403130,TOR


In [898]:
team_rows = pitchers['Pitchers'] == 'TEAM'

starting_pitcher_indices = pitchers.index[team_rows] +  1

starting_pitcher_indices = starting_pitcher_indices.insert(0, 0)

starting_pitcher_indices = starting_pitcher_indices[starting_pitcher_indices < len(pitchers)]

starting_pitchers = pitchers.loc[starting_pitcher_indices].reset_index(drop=True)

starting_pitchers.head()
starting_pitchers.shape

(26868, 4)

In [899]:
pitchers_df = starting_pitchers.rename(columns={
    'Pitchers': 'pitcher_name',
    'Pitcher Id': 'pitcher_id',
    'Game': 'game_id',
    'Team': 'team'
})

df = df.rename(columns={
    'Game': 'game_id',
    'away': 'away_team',
    'home': 'home_team'
})

# Merge away pitcher
merged = df.merge(
    pitchers_df,
    left_on=['game_id', 'away_team'],
    right_on=['game_id', 'team'],
    how='left'
).rename(columns={
    'pitcher_name': 'away_pitcher',
    'pitcher_id': 'away_pitcher_id'
}).drop(columns=['team'])

# Merge home pitcher
merged = merged.merge(
    pitchers_df,
    left_on=['game_id', 'home_team'],
    right_on=['game_id', 'team'],
    how='left'
).rename(columns={
    'pitcher_name': 'home_pitcher',
    'pitcher_id': 'home_pitcher_id'
}).drop(columns=['team'])

#Reorder columns for clarity
merged = merged[[
    'game_id', 'NRFI',
    'away_team', 'away_pitcher', 'away_pitcher_id',
    'home_team', 'home_pitcher', 'home_pitcher_id','year'
]]

df = merged
df.head()


,game_id,NRFI,away_team,away_pitcher,away_pitcher_id,home_team,home_pitcher,home_pitcher_id,year
0,360403107,False,NYM,M. Harvey,31214,KC,E. Volquez,6401,2016
1,360403123,True,STL,A. Wainwright,5403,PIT,F. Liriano,6211,2016
2,360403130,False,TOR,M. Stroman,32815,TB,C. Archer,31003,2016
3,360404101,True,MIN,E. Santana,6280,BAL,C. Tillman,30285,2016
4,360404103,False,CHC,J. Arrieta,30145,LAA,G. Richards,30892,2016


In [900]:
pitcher_stats = pd.read_csv("no_filter_pitcher_stats.csv")
pitcher_stats.shape

(4437, 15)

# must drop the accent on players names smh
# and deal with R.A. Dickey smh
# and Daniel Lynch IV smh

In [ ]:
import unicodedata

suffixes_to_remove = {'II', 'III', 'IV', 'V'}

def standardizeName(name):
    # Split "Last Suffix, First" into "Last", "First"
    last_part, first = name.split(", ")
    
    last_parts = last_part.split(" ")
    
    # Remove suffix if it's in the list
    if last_parts[-1] in suffixes_to_remove:
        last_name = " ".join(last_parts[:-1])
    else:
        last_name = last_part
        
    if "." in first:  # deal with names such as R.A. Dickey
        return first + " " + last_name

    first_initial = first[0] + "."  # Get "B." from "Buster"
    return f"{first_initial} {last_name}"


def remove_accents(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text

pitcher_stats["last_name, first_name"] = pitcher_stats["last_name, first_name"].apply(standardizeName)

pitcher_stats.rename(columns={"last_name, first_name" : "pitcher_name"}, inplace=True)

pitcher_stats['pitcher_name'] = pitcher_stats['pitcher_name'].apply(remove_accents)

pitcher_stats.head()

UnboundLocalError: cannot access local variable 'last_name' where it is not associated with a value

In [ ]:
home_stats = pitcher_stats.rename(columns=lambda col: f'home_pitcher_{col}' if col not in ['pitcher_name', 'year'] else col)
away_stats = pitcher_stats.rename(columns=lambda col: f'away_pitcher_{col}' if col not in ['pitcher_name', 'year'] else col)

df = df.merge(
    home_stats,
    left_on=['home_pitcher', 'year'],
    right_on=['pitcher_name', 'year'],
    how='left'
).drop(columns=['pitcher_name','home_pitcher_id','home_pitcher_player_id'])

df = df.merge(
    away_stats,
    left_on=['away_pitcher', 'year'],
    right_on=['pitcher_name', 'year'],
    how='left'
).drop(columns=['pitcher_name','away_pitcher_id','away_pitcher_player_id'])

pd.set_option('display.max_rows', None)
df.head()
df.shape



(14536, 31)

In [ ]:
df.isna().any(axis=1).sum()

64

# will probably just drop the 64 games, not a significant difference

In [ ]:
df['home_pitcher_low_sample'] = df['home_pitcher_pa'] < 100
df['away_pitcher_low_sample'] = df['away_pitcher_pa'] < 100
df.head()


,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample
0,360403107,False,NYM,M. Harvey,KC,E. Volquez,2016,853.0,16.3,8.9,0.343,0.347,32.2,6.3,35.5,78.467345,93.710161,19.8,45.8,402.0,18.9,6.2,0.341,0.315,38.0,4.3,37.0,77.798122,93.658710,21.8,50.9,False,False
1,360403123,True,STL,A. Wainwright,PIT,F. Liriano,2016,731.0,23.0,11.6,0.336,0.320,28.6,6.4,33.3,78.130932,93.357774,28.2,43.3,847.0,19.0,7.0,0.335,0.329,37.5,5.9,30.2,76.527704,92.663444,20.2,44.1,False,False
2,360403130,False,TOR,M. Stroman,TB,C. Archer,2016,850.0,27.4,7.9,0.304,0.302,29.7,8.4,40.5,80.179654,94.340096,29.6,45.5,854.0,19.4,6.3,0.312,0.306,26.5,4.9,41.9,79.928197,94.684735,21.5,47.3,False,False
3,360404101,True,MIN,E. Santana,BAL,C. Tillman,2016,715.0,19.6,9.2,0.317,0.330,34.7,7.8,35.3,79.233002,93.500010,21.9,45.7,748.0,19.9,7.1,0.296,0.313,32.8,5.9,31.7,78.073855,93.358088,22.9,47.2,False,False
4,360404103,False,CHC,J. Arrieta,LAA,G. Richards,2016,148.0,23.0,10.1,0.300,0.323,34.7,5.1,38.8,78.102554,94.008940,24.0,46.8,795.0,23.9,9.6,0.261,0.291,30.8,4.8,29.6,76.361267,92.514069,25.2,45.6,False,False


# Add weighted stats here

In [ ]:
batters_df = pd.read_csv("hittersByGame.csv")
batters_df.head()

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_13818/1604873123.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  batters_df = pd.read_csv("hittersByGame.csv")


,Hitters,H-AB,AB,R,H,RBI,BB,K,#P,AVG,OBP,SLG,Game,Team,Position,Hitter Id
0,M. Carpenter,1-4,4,0,1,1,1,0,22,0.25,0.4,0.25,360403123,STL,3B,31015
1,T. Pham,0-1,1,0,0,0,0,0,1,0.0,0.0,0.0,360403123,STL,LF,31208
2,M. Adams,0-4,4,0,0,0,0,2,18,0.0,0.0,0.0,360403123,STL,1B,31359
3,M. Holliday,1-3,3,0,1,0,1,1,15,0.33299999999999996,0.5,0.33299999999999996,360403123,STL,1B-LF,5940
4,R. Grichuk,1-4,4,0,1,0,0,3,14,0.25,0.25,0.25,360403123,STL,CF,31399


In [ ]:
batters_df = batters_df[["Hitters", "Game", "Team","Position","AB","BB"]]
batters_df.rename(columns={"Hitters" : "batter_name", "Game" : "game_id", }, inplace=True)
batters_df.head()

,batter_name,game_id,Team,Position,AB,BB
0,M. Carpenter,360403123,STL,3B,4,1
1,T. Pham,360403123,STL,LF,1,0
2,M. Adams,360403123,STL,1B,4,0
3,M. Holliday,360403123,STL,1B-LF,3,1
4,R. Grichuk,360403123,STL,CF,4,0


# Drop pitchers before grabbing top 5, they snuck in there

In [ ]:
batters_df = batters_df[batters_df['Position'] != 'P']

## Keep top 5 batters in the order
- top 5 is infiltrated by pinch hitters, must weed them out
- don't have the data to calculate total AB's so cant use that (for example McCutchen bats 3rd but was left off because he had one less AB + BB compared to players after him, likely from a sac fly)
- also tommy pham bat 2nd but was subbed out after one AB so would be left off
- no way of being totally confident on this

- lets just drop all guys with <= 1 AB + BB as that would be unlikely they were in the starting order

In [ ]:
batters_df['AB'] = pd.to_numeric(batters_df['AB'], errors='coerce')
batters_df['BB'] = pd.to_numeric(batters_df['BB'], errors='coerce')

batters_df = batters_df[batters_df['AB'] + batters_df['BB'] > 1]


In [ ]:
batters_df['batter_order'] = batters_df.groupby(['game_id', 'Team']).cumcount() + 1
stl = batters_df[batters_df["Team"] == "STL"]
stl.head()

,batter_name,game_id,Team,Position,AB,BB,batter_order
0,M. Carpenter,360403123,STL,3B,4.0,1.0,1
2,M. Adams,360403123,STL,1B,4.0,0.0,2
3,M. Holliday,360403123,STL,1B-LF,3.0,1.0,3
4,R. Grichuk,360403123,STL,CF,4.0,0.0,4
5,S. Piscotty,360403123,STL,RF,3.0,1.0,5


In [ ]:
top_n = 5
top_batters = batters_df[batters_df['batter_order'] <= top_n]

batters_df = top_batters.pivot(
    index=['game_id', 'Team'],
    columns='batter_order',
    values='batter_name'
).reset_index()

batters_df.head()


batter_order,game_id,Team,1,2,3,4,5
0,360403107,KC,A. Escobar,M. Moustakas,L. Cain,E. Hosmer,K. Morales
1,360403107,NYM,C. Granderson,D. Wright,Y. Cespedes,L. Duda,N. Walker
2,360403123,PIT,J. Jaso,A. McCutchen,D. Freese,S. Marte,F. Cervelli
3,360403123,STL,M. Carpenter,M. Adams,M. Holliday,R. Grichuk,S. Piscotty
4,360403130,TB,L. Forsythe,L. Morrison,E. Longoria,C. Dickerson,D. Jennings


In [ ]:
batters_df.columns = ['game_id', 'Team'] + [f'batter_{i}' for i in range(1, top_n + 1)]
batters_df.head()


,game_id,Team,batter_1,batter_2,batter_3,batter_4,batter_5
0,360403107,KC,A. Escobar,M. Moustakas,L. Cain,E. Hosmer,K. Morales
1,360403107,NYM,C. Granderson,D. Wright,Y. Cespedes,L. Duda,N. Walker
2,360403123,PIT,J. Jaso,A. McCutchen,D. Freese,S. Marte,F. Cervelli
3,360403123,STL,M. Carpenter,M. Adams,M. Holliday,R. Grichuk,S. Piscotty
4,360403130,TB,L. Forsythe,L. Morrison,E. Longoria,C. Dickerson,D. Jennings


In [ ]:
pd.set_option('display.max_columns', None)
# Merge home batters
df = df.merge(
    batters_df,
    left_on=['game_id', 'home_team'],
    right_on=['game_id', 'Team'],
    how='left',
    suffixes=('', '_home')
).drop(columns=['Team'])

df.rename(columns={f'batter_{i}': f'home_batter_{i}' for i in range(1, top_n + 1)}, inplace=True)

df = df.merge(
    batters_df,
    left_on=['game_id', 'away_team'],
    right_on=['game_id', 'Team'],
    how='left',
    suffixes=('', '_away')
).drop(columns=['Team'])

# Rename batter columns for away team
df.rename(columns={f'batter_{i}': f'away_batter_{i}' for i in range(1, top_n + 1)}, inplace=True)

df.head(n=20)


,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5
0,360403107,False,NYM,M. Harvey,KC,E. Volquez,2016,853.0,16.3,8.9,0.343,0.347,32.2,6.3,35.5,78.467345,93.710161,19.8,45.8,402.0,18.9,6.2,0.341,0.315,38.0,4.3,37.0,77.798122,93.658710,21.8,50.9,False,False,A. Escobar,M. Moustakas,L. Cain,E. Hosmer,K. Morales,C. Granderson,D. Wright,Y. Cespedes,L. Duda,N. Walker
1,360403123,True,STL,A. Wainwright,PIT,F. Liriano,2016,731.0,23.0,11.6,0.336,0.320,28.6,6.4,33.3,78.130932,93.357774,28.2,43.3,847.0,19.0,7.0,0.335,0.329,37.5,5.9,30.2,76.527704,92.663444,20.2,44.1,False,False,J. Jaso,A. McCutchen,D. Freese,S. Marte,F. Cervelli,M. Carpenter,M. Adams,M. Holliday,R. Grichuk,S. Piscotty
2,360403130,False,TOR,M. Stroman,TB,C. Archer,2016,850.0,27.4,7.9,0.304,0.302,29.7,8.4,40.5,80.179654,94.340096,29.6,45.5,854.0,19.4,6.3,0.312,0.306,26.5,4.9,41.9,79.928197,94.684735,21.5,47.3,False,False,L. Forsythe,L. Morrison,E. Longoria,C. Dickerson,D. Jennings,K. Pillar,J. Donaldson,J. Bautista,E. Encarnacion,T. Tulowitzki
3,360404101,True,MIN,E. Santana,BAL,C. Tillman,2016,715.0,19.6,9.2,0.317,0.330,34.7,7.8,35.3,79.233002,93.500010,21.9,45.7,748.0,19.9,7.1,0.296,0.313,32.8,5.9,31.7,78.073855,93.358088,22.9,47.2,False,False,M. Machado,A. Jones,C. Davis,M. Trumbo,M. Wieters,B. Dozier,J. Mauer,M. Sano,T. Plouffe,E. Rosario
4,360404103,False,CHC,J. Arrieta,LAA,G. Richards,2016,148.0,23.0,10.1,0.300,0.323,34.7,5.1,38.8,78.102554,94.008940,24.0,46.8,795.0,23.9,9.6,0.261,0.291,30.8,4.8,29.6,76.361267,92.514069,25.2,45.6,False,False,Y. Escobar,D. Nava,M. Trout,A. Pujols,K. Calhoun,D. Fowler,J. Heyward,B. Zobrist,A. Rizzo,K. Bryant
5,360404108,False,SF,M. Bumgarner,MIL,W. Peralta,2016,554.0,16.8,7.8,0.365,0.345,34.9,6.7,35.9,78.876811,93.881343,21.0,44.5,912.0,27.5,5.9,0.269,0.279,29.5,6.7,31.2,78.651695,93.234361,25.9,48.1,False,False,D. Santana,J. Villar,R. Braun,J. Lucroy,C. Carter,D. Span,J. Panik,B. Posey,H. Pence,B. Belt
6,360404108,False,SF,M. Bumgarner,MIL,W. Peralta,2016,39.0,12.8,17.9,0.447,0.386,30.8,3.8,26.9,72.305866,91.868283,33.9,42.1,912.0,27.5,5.9,0.269,0.279,29.5,6.7,31.2,78.651695,93.234361,25.9,48.1,True,False,D. Santana,J. Villar,R. Braun,J. Lucroy,C. Carter,D. Span,J. Panik,B. Posey,H. Pence,B. Belt
7,360404111,True,CHW,C. Sale,OAK,R. Hill,2016,439.0,29.4,7.5,0.241,0.259,27.9,3.7,32.3,76.354306,92.906126,28.1,42.8,907.0,25.7,5.0,0.282,0.294,33.5,5.7,33.3,78.974300,93.259391,24.8,49.3,False,False,B. Burns,J. Lowrie,J. Reddick,D. Valencia,K. Davis,A. Eaton,J. Rollins,J. Abreu,T. Frazier,M. Cabrera
8,360404113,False,SEA,F. Hernandez,TEX,C. Hamels,2016,848.0,23.6,9.1,0.307,0.315,32.0,6.0,35.7,75.943485,93.704193,27.2,48.7,655.0,18.6,9.9,0.316,0.345,31.7,5.9,34.3,78.843843,93.634040,22.5,46.6,False,False,D. DeShields,S. Choo,P. Fielder,A. Beltre,M. Moreland,N. Aoki,K. Marte,R. Cano,N. Cruz,K. Seager
9,360404115,False,WSH,M. Scherzer,ATL,J. Teheran,2016,758.0,22.0,5.4,0.281,0.298,33.1,6.5,33.6,78.801190,93.514895,24.1,48.6,902.0,31.5,6.2,0.267,0.259,30.8,6.1,29.5,76.600745,92.906857,32.4,51.8,False,False,E. Inciarte,E. Aybar,F. Freeman,A. Garcia,N. Markakis,B. Revere,M. Taylor,A. Rendon,B. Harper,R. Zimmerman


# Keep default baseball savant stats for now, can experiment with more stats later

In [ ]:
batter_stats = pd.read_csv("batter_stats.csv")
batter_stats.head()

,"last_name, first_name",player_id,year,pa,k_percent,bb_percent,woba,xwoba,sweet_spot_percent,barrel_batted_rate,hard_hit_percent,avg_best_speed,avg_hyper_speed,whiff_percent,swing_percent
0,"Colon, Bartolo",112526,2016,65,61.5,1.5,0.114,0.083,20.8,0.0,16.7,92.769404,91.325306,47.6,44.4
1,"Ortiz, David",120074,2016,626,13.7,12.8,0.419,0.430,39.3,15.5,47.2,103.045534,96.143184,19.4,43.5
2,"Rodriguez, Alex",121347,2016,243,27.6,5.8,0.258,0.287,25.5,7.5,38.5,100.783814,94.587312,29.6,49.1
3,"Beltré, Adrian",134181,2016,640,10.3,7.5,0.371,0.366,34.2,8.5,40.4,99.676160,94.135211,15.2,48.7
4,"Beltrán, Carlos",136860,2016,593,17.0,5.9,0.358,0.320,32.5,6.8,38.5,100.288108,94.247053,19.6,45.6


# Savant puts both pitcher and batter data in this csv for some reason, when we merge only batters will remain

In [ ]:
batter_stats["last_name, first_name"] = batter_stats["last_name, first_name"].apply(standardizeName)

batter_stats.rename(columns={"last_name, first_name" : "batter_name"}, inplace=True)

batter_stats['batter_name'] = batter_stats['batter_name'].apply(remove_accents)

batter_stats.head()
batter_stats.isna().any(axis=1).sum()

2

In [ ]:
for team in ['home', 'away']:
    for i in range(1, 6):
        batter_col = f'{team}_batter_{i}'
        stats = batter_stats.rename(
            columns=lambda col: f'{batter_col}_{col}' if col not in ['batter_name', 'year'] else col
        )

        df = df.merge(
            stats,
            left_on=[batter_col, 'year'],
            right_on=['batter_name', 'year'],
            how='left'
        ).drop(columns=['batter_name'])

df = df.drop_duplicates(subset='game_id').reset_index(drop=True)

df.head(n=100)



,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_pa,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_woba,home_batter_1_xwoba,home_batter_1_sweet_spot_percent,home_batter_1_barrel_batted_rate,home_batter_1_hard_hit_percent,home_batter_1_avg_best_speed,home_batter_1_avg_hyper_speed,home_batter_1_whiff_percent,home_batter_1_swing_percent,home_batter_2_player_id,home_batter_2_pa,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_woba,home_batter_2_xwoba,home_batter_2_sweet_spot_percent,home_batter_2_barrel_batted_rate,home_batter_2_hard_hit_percent,home_batter_2_avg_best_speed,home_batter_2_avg_hyper_speed,home_batter_2_whiff_percent,home_batter_2_swing_percent,home_batter_3_player_id,home_batter_3_pa,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_woba,home_batter_3_xwoba,home_batter_3_sweet_spot_percent,home_batter_3_barrel_batted_rate,home_batter_3_hard_hit_percent,home_batter_3_avg_best_speed,home_batter_3_avg_hyper_speed,home_batter_3_whiff_percent,home_batter_3_swing_percent,home_batter_4_player_id,home_batter_4_pa,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_woba,home_batter_4_xwoba,home_batter_4_sweet_spot_percent,home_batter_4_barrel_batted_rate,home_batter_4_hard_hit_percent,home_batter_4_avg_best_speed,home_batter_4_avg_hyper_speed,home_batter_4_whiff_percent,home_batter_4_swing_percent,home_batter_5_player_id,home_batter_5_pa,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_woba,home_batter_5_xwoba,home_batter_5_sweet_spot_percent,home_batter_5_barrel_batted_rate,home_batter_5_hard_hit_percent,home_batter_5_avg_best_speed,home_batter_5_avg_hyper_speed,home_batter_5_whiff_percent,home_batter_5_swing_percent,away_batter_1_player_id,away_batter_1_pa,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_woba,away_batter_1_xwoba,away_batter_1_sweet_spot_percent,away_batter_1_barrel_batted_rate,away_batter_1_hard_hit_percent,away_batter_1_avg_best_speed,away_batter_1_avg_hyper_speed,away_batter_1_whiff_percent,away_batter_1_swing_percent,away_batter_2_player_id,away_batter_2_pa,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_woba,away_batter_2_xwoba,away_batter_2_sweet_spot_percent,away_batter_2_barrel_batted_rate,away_batter_2_hard_hit_percent,away_batter_2_avg_best_speed,away_batter_2_avg_hyper_speed,away_batter_2_whiff_percent,away_batter_2_swing_percent,away_batter_3_player_id,away_batter_3_pa,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_woba,away_batter_3_xwoba,away_batter_3_sweet_spot_percent,away_batter_3_barrel_batted_rate,away_batter_3_hard_hit_percent,away_batter_3_avg_best_speed,away_batter_3_avg_hyper_speed,away_batter_3_whiff_percent,away_batter_3_swing_percent,away_batter_4_player_id,away_batter_4_pa,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_woba,away_batter_4_xwoba,away_batter_4_sweet_spot_percent,away_batter_4_barrel_batted_rate,away_batter_4_hard_hit_percent,away_batter_4_avg_best_speed,away_batter_4_avg_hyper_speed,away_batter_4_whiff_percent,away_batter_4_swing_percent,away_batter_5_player_id,away_batter_5_pa,away_batter_5_k_percent,awa

In [ ]:
batter_stats_to_agg = ['xwoba', 'woba', 'k_percent', 'bb_percent', 'barrel_batted_rate','sweet_spot_percent','hard_hit_percent','avg_best_speed','avg_hyper_speed','whiff_percent','swing_percent']

for stat in batter_stats_to_agg:
    df[f'home_top5_avg_{stat}'] = df[[f'home_batter_{i}_{stat}' for i in range(1, 6)]].mean(axis=1)
    df[f'away_top5_avg_{stat}'] = df[[f'away_batter_{i}_{stat}' for i in range(1, 6)]].mean(axis=1)
    
df.head()


,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_pa,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_woba,home_batter_1_xwoba,home_batter_1_sweet_spot_percent,home_batter_1_barrel_batted_rate,home_batter_1_hard_hit_percent,home_batter_1_avg_best_speed,home_batter_1_avg_hyper_speed,home_batter_1_whiff_percent,home_batter_1_swing_percent,home_batter_2_player_id,home_batter_2_pa,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_woba,home_batter_2_xwoba,home_batter_2_sweet_spot_percent,home_batter_2_barrel_batted_rate,home_batter_2_hard_hit_percent,home_batter_2_avg_best_speed,home_batter_2_avg_hyper_speed,home_batter_2_whiff_percent,home_batter_2_swing_percent,home_batter_3_player_id,home_batter_3_pa,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_woba,home_batter_3_xwoba,home_batter_3_sweet_spot_percent,home_batter_3_barrel_batted_rate,home_batter_3_hard_hit_percent,home_batter_3_avg_best_speed,home_batter_3_avg_hyper_speed,home_batter_3_whiff_percent,home_batter_3_swing_percent,home_batter_4_player_id,home_batter_4_pa,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_woba,home_batter_4_xwoba,home_batter_4_sweet_spot_percent,home_batter_4_barrel_batted_rate,home_batter_4_hard_hit_percent,home_batter_4_avg_best_speed,home_batter_4_avg_hyper_speed,home_batter_4_whiff_percent,home_batter_4_swing_percent,home_batter_5_player_id,home_batter_5_pa,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_woba,home_batter_5_xwoba,home_batter_5_sweet_spot_percent,home_batter_5_barrel_batted_rate,home_batter_5_hard_hit_percent,home_batter_5_avg_best_speed,home_batter_5_avg_hyper_speed,home_batter_5_whiff_percent,home_batter_5_swing_percent,away_batter_1_player_id,away_batter_1_pa,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_woba,away_batter_1_xwoba,away_batter_1_sweet_spot_percent,away_batter_1_barrel_batted_rate,away_batter_1_hard_hit_percent,away_batter_1_avg_best_speed,away_batter_1_avg_hyper_speed,away_batter_1_whiff_percent,away_batter_1_swing_percent,away_batter_2_player_id,away_batter_2_pa,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_woba,away_batter_2_xwoba,away_batter_2_sweet_spot_percent,away_batter_2_barrel_batted_rate,away_batter_2_hard_hit_percent,away_batter_2_avg_best_speed,away_batter_2_avg_hyper_speed,away_batter_2_whiff_percent,away_batter_2_swing_percent,away_batter_3_player_id,away_batter_3_pa,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_woba,away_batter_3_xwoba,away_batter_3_sweet_spot_percent,away_batter_3_barrel_batted_rate,away_batter_3_hard_hit_percent,away_batter_3_avg_best_speed,away_batter_3_avg_hyper_speed,away_batter_3_whiff_percent,away_batter_3_swing_percent,away_batter_4_player_id,away_batter_4_pa,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_woba,away_batter_4_xwoba,away_batter_4_sweet_spot_percent,away_batter_4_barrel_batted_rate,away_batter_4_hard_hit_percent,away_batter_4_avg_best_speed,away_batter_4_avg_hyper_speed,away_batter_4_whiff_percent,away_batter_4_swing_percent,away_batter_5_player_id,away_batter_5_pa,away_batter_5_k_percent,awa

# Let's start to train our data, we can add stats to optimize later

- recent performance
- specific performance (how hitters perform against specific pitchers)
- weather + stadium
- weighted stats based on sample size

## first lets get overall team nrfi percentage as that must be a big indicator

Going to scrape teamrankings.com for the data


In [ ]:
%pip install html5lib

import pandas as pd
import time, random, os

# Load data if already scraped
if os.path.exists('first-inning-runs-per-game.csv'):
    first_inning_rpg_df = pd.read_csv('first-inning-runs-per-game.csv', index_col=0)
else:
    # List of seasons to scrape
    seasons = ['2016', '2017', '2018', '2019', '2020', '2021']
    first_inning_rpg_df = pd.DataFrame()

    for season in seasons:
        url = f'https://www.teamrankings.com/mlb/stat/1st-inning-runs-per-game?date={season}-11-10'
        
        try:
            tables = pd.read_html(url, header=0)
            if tables:
                season_df = tables[0]
                season_df['Season'] = season  # Add season column
                first_inning_rpg_df = pd.concat([first_inning_rpg_df, season_df], ignore_index=True)
                print(f"Scraped season {season}")
            else:
                print(f"No table found for season {season}")
        except Exception as e:
            print(f"Failed to scrape season {season}: {e}")
        
        time.sleep(random.randint(4, 6))  # Sleep to avoid rate limiting

    # Save the data
    first_inning_rpg_df.to_csv('first-inning-runs-per-game.csv')



Note: you may need to restart the kernel to use updated packages.


In [ ]:
first_inning_rpg_df.head()

,Rank,Team,2016,Last 3,Last 1,Home,Away,2015,Season,2017,2018,2019,2020,2021
0,1,Washington,0.72,0.33,0.0,0.62,0.83,0.55,2016,NaN,NaN,NaN,NaN,NaN
1,2,Boston,0.71,0.33,0.0,0.89,0.53,0.48,2016,NaN,NaN,NaN,NaN,NaN
2,3,Cincinnati,0.68,1.33,3.0,0.73,0.63,0.59,2016,NaN,NaN,NaN,NaN,NaN
3,4,Chi Cubs,0.67,1.33,1.0,0.69,0.66,0.53,2016,NaN,NaN,NaN,NaN,NaN
4,5,Arizona,0.65,2.00,0.0,0.72,0.59,0.52,2016,NaN,NaN,NaN,NaN,NaN


In [ ]:
rows = []

years = [2016, 2017, 2018, 2019, 2020, 2021]
column_indices = {2016: 2, 2017: 9, 2018: 10, 2019: 11, 2020: 12, 2021: 13}

for year in years:
    col_idx = column_indices[year]

    values = first_inning_rpg_df[first_inning_rpg_df[str(year)].notna()].iloc[:30, [1, col_idx]]
    
    for _, row in values.iterrows():
        rows.append({
            "year": year,
            "team": row.iloc[0],    
            "rpg": row.iloc[1]
        })

first_inning_rpg_df = pd.DataFrame(rows)
first_inning_rpg_df.head()


,year,team,rpg
0,2016,Washington,0.72
1,2016,Boston,0.71
2,2016,Cincinnati,0.68
3,2016,Chi Cubs,0.67
4,2016,Arizona,0.65


# Just use overall for now, can optimize with home / away splits

# See what abbreviations and team names we are dealing with

- so we know how to translate
- deal with relocations (oakland to sacramento)

In [ ]:
print(first_inning_rpg_df["team"].drop_duplicates().tolist())

['Washington', 'Boston', 'Cincinnati', 'Chi Cubs', 'Arizona', 'Pittsburgh', 'Toronto', 'LA Dodgers', 'Colorado', 'LA Angels', 'Seattle', 'Houston', 'Miami', 'Texas', 'Milwaukee', 'Tampa Bay', 'San Diego', 'Cleveland', 'Baltimore', 'Kansas City', 'Chi Sox', 'Detroit', 'Minnesota', 'NY Mets', 'NY Yankees', 'St. Louis', 'Philadelphia', 'Atlanta', 'SF Giants', 'Sacramento']


In [ ]:
print(df["away_team"].drop_duplicates().tolist())

df[df["away_team"] == 'AL']


['NYM', 'STL', 'TOR', 'MIN', 'CHC', 'SF', 'CHW', 'SEA', 'WSH', 'PHI', 'LAD', 'COL', 'BOS', 'HOU', 'DET', 'TEX', 'MIA', 'TB', 'CLE', 'NYY', 'OAK', 'PIT', 'SD', 'BAL', 'LAA', 'CIN', 'KC', 'ATL', 'MIL', 'ARI', 'NL', 'AL']


,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_pa,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_woba,home_batter_1_xwoba,home_batter_1_sweet_spot_percent,home_batter_1_barrel_batted_rate,home_batter_1_hard_hit_percent,home_batter_1_avg_best_speed,home_batter_1_avg_hyper_speed,home_batter_1_whiff_percent,home_batter_1_swing_percent,home_batter_2_player_id,home_batter_2_pa,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_woba,home_batter_2_xwoba,home_batter_2_sweet_spot_percent,home_batter_2_barrel_batted_rate,home_batter_2_hard_hit_percent,home_batter_2_avg_best_speed,home_batter_2_avg_hyper_speed,home_batter_2_whiff_percent,home_batter_2_swing_percent,home_batter_3_player_id,home_batter_3_pa,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_woba,home_batter_3_xwoba,home_batter_3_sweet_spot_percent,home_batter_3_barrel_batted_rate,home_batter_3_hard_hit_percent,home_batter_3_avg_best_speed,home_batter_3_avg_hyper_speed,home_batter_3_whiff_percent,home_batter_3_swing_percent,home_batter_4_player_id,home_batter_4_pa,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_woba,home_batter_4_xwoba,home_batter_4_sweet_spot_percent,home_batter_4_barrel_batted_rate,home_batter_4_hard_hit_percent,home_batter_4_avg_best_speed,home_batter_4_avg_hyper_speed,home_batter_4_whiff_percent,home_batter_4_swing_percent,home_batter_5_player_id,home_batter_5_pa,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_woba,home_batter_5_xwoba,home_batter_5_sweet_spot_percent,home_batter_5_barrel_batted_rate,home_batter_5_hard_hit_percent,home_batter_5_avg_best_speed,home_batter_5_avg_hyper_speed,home_batter_5_whiff_percent,home_batter_5_swing_percent,away_batter_1_player_id,away_batter_1_pa,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_woba,away_batter_1_xwoba,away_batter_1_sweet_spot_percent,away_batter_1_barrel_batted_rate,away_batter_1_hard_hit_percent,away_batter_1_avg_best_speed,away_batter_1_avg_hyper_speed,away_batter_1_whiff_percent,away_batter_1_swing_percent,away_batter_2_player_id,away_batter_2_pa,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_woba,away_batter_2_xwoba,away_batter_2_sweet_spot_percent,away_batter_2_barrel_batted_rate,away_batter_2_hard_hit_percent,away_batter_2_avg_best_speed,away_batter_2_avg_hyper_speed,away_batter_2_whiff_percent,away_batter_2_swing_percent,away_batter_3_player_id,away_batter_3_pa,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_woba,away_batter_3_xwoba,away_batter_3_sweet_spot_percent,away_batter_3_barrel_batted_rate,away_batter_3_hard_hit_percent,away_batter_3_avg_best_speed,away_batter_3_avg_hyper_speed,away_batter_3_whiff_percent,away_batter_3_swing_percent,away_batter_4_player_id,away_batter_4_pa,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_woba,away_batter_4_xwoba,away_batter_4_sweet_spot_percent,away_batter_4_barrel_batted_rate,away_batter_4_hard_hit_percent,away_batter_4_avg_best_speed,away_batter_4_avg_hyper_speed,away_batter_4_whiff_percent,away_batter_4_swing_percent,away_batter_5_player_id,away_batter_5_pa,away_batter_5_k_percent,awa

# Realized that some all star games are in here
## lets get rid of those

In [ ]:
df = df[~df["away_team"].isin(['AL', 'NL'])]
print(df["away_team"].drop_duplicates().tolist())
print(len(df["away_team"].drop_duplicates().tolist()))

['NYM', 'STL', 'TOR', 'MIN', 'CHC', 'SF', 'CHW', 'SEA', 'WSH', 'PHI', 'LAD', 'COL', 'BOS', 'HOU', 'DET', 'TEX', 'MIA', 'TB', 'CLE', 'NYY', 'OAK', 'PIT', 'SD', 'BAL', 'LAA', 'CIN', 'KC', 'ATL', 'MIL', 'ARI']
30


In [ ]:
location_to_abbrev = {
    'St. Louis': 'STL',
    'NY Yankees': 'NYY',
    'NY Mets': 'NYM',
    'Chi Sox': 'CHW',
    'Chi Cubs': 'CHC',
    'LA Dodgers': 'LAD',
    'LA Angels': 'LAA',
    'Boston': 'BOS',
    'Atlanta': 'ATL',
    'Cincinnati': 'CIN',
    'Cleveland': 'CLE',
    'Colorado': 'COL',
    'Detroit': 'DET',
    'Houston': 'HOU',
    'Kansas City': 'KC',
    'Miami': 'MIA',
    'Milwaukee': 'MIL',
    'Minnesota': 'MIN',
    'Sacramento': 'OAK',  # teamrankings.com updated to sacramento while our data had not
    'Philadelphia': 'PHI',
    'Pittsburgh': 'PIT',
    'San Diego': 'SD',
    'SF Giants': 'SF',
    'Seattle': 'SEA',
    'Tampa Bay': 'TB',
    'Texas': 'TEX',
    'Toronto': 'TOR',
    'Washington': 'WSH',
    'Baltimore' : 'BAL',
    'Arizona' : 'ARI'
}

# Convert the 'Location' column to abbreviations
first_inning_rpg_df['team'] = first_inning_rpg_df['team'].map(location_to_abbrev)

first_inning_rpg_df.head()



,year,team,rpg
0,2016,WSH,0.72
1,2016,BOS,0.71
2,2016,CIN,0.68
3,2016,CHC,0.67
4,2016,ARI,0.65


In [ ]:
df = df.merge(
    first_inning_rpg_df,
    how='left',
    left_on=['year', 'away_team'],
    right_on=['year', 'team'],
    suffixes=('', '_away')  # Keep away stats unmodified, rename right side
).drop(columns=['team'])  # Drop duplicated 'team' column

df = df.rename(columns=lambda col: f'away_{col}' if col == 'rpg' else col)


# Merge home team stats
df = df.merge(
    first_inning_rpg_df,
    how='left',
    left_on=['year', 'home_team'],
    right_on=['year', 'team'],
    suffixes=('', '_home')
).drop(columns=['team'])

df = df.rename(columns=lambda col: f'home_{col}' if col == 'rpg' else col)


df.head()


,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_pa,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_woba,home_pitcher_xwoba,home_pitcher_sweet_spot_percent,home_pitcher_barrel_batted_rate,home_pitcher_hard_hit_percent,home_pitcher_avg_best_speed,home_pitcher_avg_hyper_speed,home_pitcher_whiff_percent,home_pitcher_swing_percent,away_pitcher_pa,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_woba,away_pitcher_xwoba,away_pitcher_sweet_spot_percent,away_pitcher_barrel_batted_rate,away_pitcher_hard_hit_percent,away_pitcher_avg_best_speed,away_pitcher_avg_hyper_speed,away_pitcher_whiff_percent,away_pitcher_swing_percent,home_pitcher_low_sample,away_pitcher_low_sample,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_pa,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_woba,home_batter_1_xwoba,home_batter_1_sweet_spot_percent,home_batter_1_barrel_batted_rate,home_batter_1_hard_hit_percent,home_batter_1_avg_best_speed,home_batter_1_avg_hyper_speed,home_batter_1_whiff_percent,home_batter_1_swing_percent,home_batter_2_player_id,home_batter_2_pa,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_woba,home_batter_2_xwoba,home_batter_2_sweet_spot_percent,home_batter_2_barrel_batted_rate,home_batter_2_hard_hit_percent,home_batter_2_avg_best_speed,home_batter_2_avg_hyper_speed,home_batter_2_whiff_percent,home_batter_2_swing_percent,home_batter_3_player_id,home_batter_3_pa,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_woba,home_batter_3_xwoba,home_batter_3_sweet_spot_percent,home_batter_3_barrel_batted_rate,home_batter_3_hard_hit_percent,home_batter_3_avg_best_speed,home_batter_3_avg_hyper_speed,home_batter_3_whiff_percent,home_batter_3_swing_percent,home_batter_4_player_id,home_batter_4_pa,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_woba,home_batter_4_xwoba,home_batter_4_sweet_spot_percent,home_batter_4_barrel_batted_rate,home_batter_4_hard_hit_percent,home_batter_4_avg_best_speed,home_batter_4_avg_hyper_speed,home_batter_4_whiff_percent,home_batter_4_swing_percent,home_batter_5_player_id,home_batter_5_pa,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_woba,home_batter_5_xwoba,home_batter_5_sweet_spot_percent,home_batter_5_barrel_batted_rate,home_batter_5_hard_hit_percent,home_batter_5_avg_best_speed,home_batter_5_avg_hyper_speed,home_batter_5_whiff_percent,home_batter_5_swing_percent,away_batter_1_player_id,away_batter_1_pa,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_woba,away_batter_1_xwoba,away_batter_1_sweet_spot_percent,away_batter_1_barrel_batted_rate,away_batter_1_hard_hit_percent,away_batter_1_avg_best_speed,away_batter_1_avg_hyper_speed,away_batter_1_whiff_percent,away_batter_1_swing_percent,away_batter_2_player_id,away_batter_2_pa,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_woba,away_batter_2_xwoba,away_batter_2_sweet_spot_percent,away_batter_2_barrel_batted_rate,away_batter_2_hard_hit_percent,away_batter_2_avg_best_speed,away_batter_2_avg_hyper_speed,away_batter_2_whiff_percent,away_batter_2_swing_percent,away_batter_3_player_id,away_batter_3_pa,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_woba,away_batter_3_xwoba,away_batter_3_sweet_spot_percent,away_batter_3_barrel_batted_rate,away_batter_3_hard_hit_percent,away_batter_3_avg_best_speed,away_batter_3_avg_hyper_speed,away_batter_3_whiff_percent,away_batter_3_swing_percent,away_batter_4_player_id,away_batter_4_pa,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_woba,away_batter_4_xwoba,away_batter_4_sweet_spot_percent,away_batter_4_barrel_batted_rate,away_batter_4_hard_hit_percent,away_batter_4_avg_best_speed,away_batter_4_avg_hyper_speed,away_batter_4_whiff_percent,away_batter_4_swing_percent,away_batter_5_player_id,away_batter_5_pa,away_batter_5_k_percent,awa

In [ ]:
df.shape

(13374, 197)

In [ ]:
df.isna().any(axis=1).sum()

3988